In [2]:
% Initial settings of the GNU Octave kernel
warning('off'); graphics_toolkit('gnuplot'); format compact;

Následujíví Latexové makro zavádí příkaz `\Aop` pro MKP operátor sestavení globálních matic:
$\newcommand{\Aop}{\mathop{\vphantom{\sum}\vphantom{\rule{0pt}{2.5ex}}\smash{\lower{1.0ex}{\text{{\huge A}}}}}}$

## Disclaimer: didaktický kód, ne produkční
- Zvolen jednoduchý programovací jazyk GNU Octave.
- Cílem je vysvětlit algoritmy na jednoduché implementaci.
- Bez snahy o solidní softwarový design.
- Bez optimalizace kódu.
- Chybí řádné ošetření chybových stavů a vyjímek.
- Proměnné a konstanty zaváděny těsně před prvním použitím.

## Řešení 2D úlohy lineární elasticity pomocí MKP
$\newcommand{\Aop}{\mathop{\vphantom{\sum}\vphantom{\rule{0pt}{2.5ex}}\smash{\lower{1.0ex}{\text{{\large A}}}}}}$
Vyjdeme ze slabé formulace (bez objemových sil):
$$
\int_{\Omega}\delta\boldsymbol{\varepsilon}:\boldsymbol{\sigma}\,\text{d}V-\int_{\Gamma_{\text{N}}}\delta\boldsymbol{u}\cdot\bar{\boldsymbol{t}}\,\text{d}S=0
$$
a její MKP diskretizace:
$$
\delta\mathbf{u}^{\mathsf{T}}
\left\{ \left[\Aop_{el=1}^{n_{\text{el}}}\int_{\Omega_{el}}\mathbf{B}^{\mathsf{T}}\mathbf{D}\mathbf{B}\,\text{d}V_{el}\right]\mathbf{u}-\Aop_{el=1}^{n_{\text{el}}}\left[\int_{\Gamma_{\text{N}}^{e}}\mathbf{N}^{\mathsf{T}}\bar{\mathbf{t}}\,\text{d}S_{el}\right]\right\} =0
$$
vede na lineární systém algebraický rovnic:
$$
\mathbf{K}\mathbf{u}-\mathbf{f}=\mathbf{0}
$$

kde:
$$
\mathbf{K}= 
\Aop_{el=1}^{n_\text{en}}
\sum_{g}\mathbf{B}_{g}^{\mathsf{T}}\mathbf{D}\mathbf{B}_{g}w_{g}\left\Vert J_{g}\right\Vert ,\qquad
\mathbf{f}=
\Aop_{el=1}^{n_\text{en}}
\sum_{g}\mathbf{N}^{\mathsf{T}}\bar{\mathbf{t}}_{g}w_{g}\left\Vert J_{g}^{\text{s}}\right\Vert 
$$
### Co budeme potřebovat:
1. Tvorba sítě
2. Sestavení globání matice a vektoru
3. Gaussova inetgrace
4. Matice tvarových funkcí $\mathbf{N}$
5. Operátorová matice $\mathbf{B}$
6. Matice materiálové tuhosti $\mathbf{D}$
7. Okrajové podmínky
8. Řešení
9. Post-processing

## 1. Tvorba sítě

Pro definovanou geometrii potřebujeme vytvořit výpočtovou síť. Síť v našém kódu budeme reprezentovat dvěma dvourozměrnými poli: polem souřadnic uzlů `X` a polem konektivity elementů `IEN`. Tyto pole můžeme pro jednoduché oblasti vytvořit ručně, ale pro složitější geometrie je užitečné použít tzv. síťovač, např. [Gmsh](https://gmsh.info).

**Prompt:**
```
Vytvoř vstupní soubor pro Gmsh `nosnik.geo`, pro 2D obdélník o délce L = 1 m a šířce b = sqrt(A) m, kde průřez A = 0.1 m^2. Počátek souřadnic je v levém horním rohu a souřadnice b míří ve směru +x a L ve směru -y. Obdélník vysíťuj třemi lineárními quad elementy ve směru y.
```

### Souřadnice uzlových bodů


In [ ]:
nsd = 2; % Number of Spatial Dimensions
run("assets/bar.m"); % read msh data structure exported from gmsh
X = msh.POS(:,1:nsd)'

### Pole konektivity


In [ ]:
nen = 4; % Number of Element Nodes
IEN = msh.QUADS(:,1:nen)'

## 1. Sestavení globání matice a vektoru

Nejprve si zvolíme, že neznámé složky posunutí pro jednotlivé uzly sítě budeme řadit ve vektoru řešní $\mathbf{u}$ takto:

$$
\begin{array}{c|ccccccccc}
i & 1 & 2 & 3 & 4 & 5 & 6 & \cdots & (n_{\text{np}}\times n_{\text{nd}})-1 & n_{\text{np}}\times n_{\text{nd}}\\
\hline
u_i & u_{1} & v_{1} & u_{2} & v_{2} & u_{3} & v_{3} & \cdots & u_{\text{nnp}} & v_{\text{nnp}}
\end{array}
$$

Vidíme, že délka vektoru $\mathbf{u}$ je součin počtu uzlů sítě `nnp` a počtu stupňů volnosti uzlu `nnd`, který je pro náš 2D případ roven 2.

Abychom se mohli snadněji odkazovat mezi globálními čísli uzlů $A\in\left[1,\ldots n_{\text{np}}\right]$ a čísly rovnic jim odpovídajícími $eq\in\left[1,\ldots n_{\text{eq}}\right]$, kde  $n_{\text{eq}}=n_{\text{np}}\times n_{\text{nd}}$, zavedeme další pomocné pole konektivity `ID` takto: 

$$
\mathtt{ID} = \left[\begin{array}{ccccc}
1 & 3 & 5 & \cdots & n_{\text{eq}} - 1 \\
2 & 4 & 6 & \ldots & n_{\text{eq}}
\end{array}\right]
$$

Vidíme, že `A`-tý sloupec pole `ID` odpovídá `A`-tému globálnímu uzlu. Řádky v tomto `A`-tém sloupci obsahují odpovídající čísla neznámých složek vektoru posunutí $\mathbf{u}$.

Pole `ID` bychom mohli implementovat pomocí cyklu a indexové aritmetiky. Pro jednoduchost však využijeme vestavěné funkci `reshape()`, která 1D pole čísel rovnic převede na 2D pole, přesně jak jsme si definovali:

In [ ]:
nnp = size(X,2);   % Number of Nodal Points
nnd = nsd;         % Number of Nodal DOFs =nsd=2(ux, uy)
neq = nnp * nnd;   % Number od EQuations
ID = reshape(1:nnp*nnd, nnd, [])

## 1. Sestavení globání matice a vektoru

Na nejvyšší úrovni se provádí proces sestavení $\Aop_{el=1}^{n_\text{en}}$ globální matice tuhosti $\mathbf{K}$ a globálního vektoru pravé strany $\mathbf{f}$ z lokálních matic $\mathbf{K}_{el}$ a lokálních vektorů $\mathbf{f}_{el}$.

Následující fragment kódu ukazuje proces sestavení jako cyklus přes všechny elementy `nel`, ve kterém se z pole konektivity `IEN` získají globální čísla úzlů `A` elementu `el`. Pro ty se následně z pole `ID` získají čísla rovnic. Protože `ID` je dvourozměrné pole, `eq` je také dvourozměrné. Abychom jej mohli použít jako řez globální maticí a vektore, použijeme opět funkci `reshape()` a převedeme jej na jednorozměrný vektor indexů. V závěru for cyklu už jen snadno **kumulujeme** příspěvky z lokální matice a vektoru.

In [ ]:
K = sparse(neq, neq);
f = zeros(neq, 1);

nel = size(IEN,2); % Number of ELement
for el = 1:nel
    Ke = zeros(nen*nnd,nen*nnd);
    fe = zeros(nen*nnd,1);
    
    A = IEN(:,el);
    
    eq = ID(:,A);
    eq = reshape(eq, [], 1);
    K(eq, eq) = K(eq, eq) + Ke;
    f(eq) = f(eq) + fe;
end

## 2. Gaussova inetgrace

Pro výpočet lokální matice tuhosti $\mathbf{K}_{el}$ potřebujeme numerickou integraci přes objem elementu. Numerická integrace spočívá v aproximaci integrálu funkce sumací jejich funkční hodnot ve vhodně vybraných bodech vynásobených váhovými koeficienty těchto bodů

$$
\int_{\square}f(\boldsymbol{\xi})\,\text{d}\boldsymbol{\xi}\approx\sum_{g=1}^{n_{\text{gp}}}f(\boldsymbol{\xi}_{g})w_{g}
$$

kde $\square\equiv\left[-1,1\right]\times\left[-1,1\right]$. Přechod od integrační oblasti $\square$ k oblasti elementu $\Omega_{el}$ souvisí s transformací isoparametrických souřadnic $\boldsymbol{\xi}$ na fyzikální souřadnice $\boldsymbol{x}$ a pozornost mu bude věnována později.

Konkrétní polohy integračních bodů a jejich váhy závisí na typu a řádu integrace. V MKP se nejčastěji používá Gaussova-Legendreova integrace. Je přesná pro polynomy až do stupně $2n=1$, kde $n$ je počet uzlů 1D integrace. Pro definici opět využijeme LLM prompt.

**Prompt:**
```
Dej mi dvě řádkové matice `gp` a `gw`, zapsané v syntaxi GNU Octave, jejichž sloupce budou odpovídat souřadnicím a vahám 2D Gaussovy- Legendreovy integrace řádu optimálního pro čtyřuzlové izoparametrické bilineární quad elementy.
```

In [ ]:
gp_bulk = [-1 1 1 -1;
           -1 -1 1 1] / sqrt(3);
gw_bulk = [1 1 1 1];
ngp = length(gw_bulk);

Zbývá zanořit cyklus přes integrační body do cyklu přes elementy. V těchto bodech potřebujeme vyčíslit tvarové funkce a jejich derivace. Jak na to si ukážeme dále.

In [ ]:
K = sparse(neq, neq);
f = zeros(neq, 1);

nel = size(IEN,2); % Number of ELement
for el = 1:nel
    Ke = zeros(nen*nnd,nen*nnd);
    fe = zeros(nen*nnd,1);

    A = IEN(:,el);
    
    for g = 1:ngp
       Xi_g = gp_bulk(:,g);
       w_g  = gw_bulk(g);
    end

    eq = ID(:,A);
    eq = reshape(eq, [], 1);
    K(eq, eq) = K(eq, eq) + Ke;
    f(eq) = f(eq) + fe;
end

## 3. Matice tvarových funkcí $\mathbf{N}$

**Prompt:**
```
Vrať mi anonymní funkce v jazyku GNU Octave `bulkShapeFunctions = @(Xi)` a `bulkShapeFunctionsDerivatives = @(Xi)` kerá pro zadané isoparametrické souřadnice Xi(1) a Xi(2) vrátí řádkové matice funkčních hodnot tvarových funkcí a jejich derivacé pro isoparametrický bilineární čtyřuzlový quad element. Lokální číslování zvol v protisměru hodinových ručiček s počátkem v levém dolním rohu.
```

In [ ]:
bulkShapeFunctions = 
    @(Xi) (1 + [-1 1 1 -1]*Xi(1)) .* (1 + [-1 -1 1 1]*Xi(2)) / 4;

bulkShapeFunctionsDerivatives = 
    @(Xi) [[-1 1 1 -1]   .* (1 + [-1 -1 1 1]*Xi(2));
      (1 + [-1 1 1 -1]*Xi(1)) .* [-1 -1 1 1]] / 4;

In [ ]:
K = sparse(neq, neq);
f = zeros(neq, 1);

nel = size(IEN,2); % Number of ELement
for el = 1:nel
    Ke = zeros(nen*nnd,nen*nnd);
    fe = zeros(nen*nnd,1);

    A = IEN(:,el);
    
    for g = 1:ngp
       Xi_g = gp_bulk(:,g);
       w_g  = gw_bulk(g);
       N_g = bulkShapeFunctions(Xi_g);
       dNdXi_g = bulkShapeFunctionsDerivatives(Xi_g);
    end

    eq = ID(:,A);
    eq = reshape(eq, [], 1);
    K(eq, eq) = K(eq, eq) + Ke;
    f(eq) = f(eq) + fe;
end

In [ ]:
for el = 1:nel
    Ke = zeros(nen*nnd,nen*nnd);
    for g = 1:ngp
        N = evalBulkShapeFunctions( gp(:,g) );
        dN_dXi = evalBulkShapeFunctionsDerivatives( gp(:,g) );
        Xe = X(:, IEN(:,el) );
        Je = Xe * dN_dXi';
        j = det(Je);
        dN_dX = inv(Je) * dN_dXi;
        Be = createMatrixB(dN_dX);

        Ke += Be' * D * Be gw(g) * j
    end
end

## Operátorová matice $\mathbf{B}$

Operátorová matice $\mathbf{B}$ zobrazí vektor posunutí $\mathbf{u}$ na vektor přetvoření $\left\{ \boldsymbol{\varepsilon}\right\} $:
$$
\left\{ \boldsymbol{\varepsilon}\right\} =\mathbf{B}\mathbf{u}
$$
Pro 2D případ

$$
\left\{ \begin{array}{c}
\varepsilon_{x}\\
\varepsilon_{y}\\
\gamma_{xy}
\end{array}\right\} =\left[\begin{array}{cc}
\frac{\partial}{\partial x} & 0\\
0 & \frac{\partial}{\partial y}\\
\frac{\partial}{\partial y} & \frac{\partial}{\partial x}
\end{array}\right]\left\{ \begin{array}{c}
u\\
v
\end{array}\right\} =\left[\begin{array}{cc}
\frac{\partial}{\partial x} & 0\\
0 & \frac{\partial}{\partial y}\\
\frac{\partial}{\partial y} & \frac{\partial}{\partial x}
\end{array}\right]\left\{ \begin{array}{c}
\sum_{a=1}^{n_{\text{en}}}N_{a}u_{a}\\
\sum_{a=1}^{n_{\text{en}}}N_{a}v_{a}
\end{array}\right\} 
$$

$$
\left\{ \begin{array}{c}
\varepsilon_{x}\\
\varepsilon_{y}\\
\gamma_{xy}
\end{array}\right\} =\left[\left[\begin{array}{cc}
\frac{\partial N_{1}}{\partial x} & 0\\
0 & \frac{\partial N_{1}}{\partial y}\\
\frac{\partial N_{1}}{\partial y} & \frac{\partial N_{1}}{\partial x}
\end{array}\right]\cdots\left[\begin{array}{cc}
\frac{\partial N_{n_{\text{en}}}}{\partial x} & 0\\
0 & \frac{\partial N_{n_{\text{en}}}}{\partial y}\\
\frac{\partial N_{n_{\text{en}}}}{\partial y} & \frac{\partial N_{n_{\text{en}}}}{\partial x}
\end{array}\right]\right]\left\{ \begin{array}{c}
u_{a}\\
v_{a}\\
\vdots\\
u_{n_{\text{en}}}\\
v_{n_{\text{en}}}
\end{array}\right\} 
$$

In [ ]:
function B = createMatrixB(dNdX)
    nsd = size(dNdX,1); % NUmber of Spatial Dimensions
    nen = size(dNdX,2); % Number of Element Nodes
    B = zeros(nsd+1, nsd*nen);
    for a = 1:nen
        B(:, (a-1)*nsd+1:a*nsd ) = [
         dNdX(1,a)     0
            0       dNdX(2,a)
         dNdX(2,a)  dNdX(2,a)];
    end
end

## Matice materiálové tuhosti $\mathbf{D}$

Matice materiálové tuhosti je lineární operátor, který vektor přetvoření $\left\{ \boldsymbol{\varepsilon}\right\}$ zobrazí na vektor napětí $\left\{ \boldsymbol{\sigma}\right\}$: 
$$
\left\{ \boldsymbol{\sigma}\right\} =\mathbf{D}\left\{ \boldsymbol{\varepsilon}\right\} 
$$
podobně jako to dělá konstanta úměrnosti (Youngův modul pružnosti) $E$ v 1D podobě Hookeova zákona: 
$$
\sigma=E\varepsilon
$$
Ve 2D je potřeba rozlišovat mezi případem rovinné **deformace** a rovinné **napjatosti** viz poznámky nebo následující prompt pro LLM.

**Prompt:**
```
Dej mi matici materiálové tuhosti D pro případ rovinné deformace zapsanou v syntaxi GNU Octave.
```

In [ ]:
E = 210e9;   % Youngův modul [Pa]
nu = 0.3;    % Poissonovo číslo [-]
D = E / ((1 + nu) * (1 - 2*nu)) * [1-nu,  nu,       0;
                                   nu,    1-nu,     0;
                                   0,     0,    (1-2*nu)/2];